In [ ]:
import pandas as pd
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [ ]:
df = pd.read_csv("/data/Churn_Modelling.csv")
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)


In [ ]:
def apply_transformers(df, categorical_columns, columns_to_scale):
    transformer = make_column_transformer(
        (StandardScaler(), columns_to_scale),
        (OneHotEncoder(sparse_output=False, drop='if_binary'), categorical_columns),
        remainder='passthrough',
        verbose_feature_names_out=False
    ).set_output(transform="pandas")

    transformed_df = transformer.fit_transform(df)

    return transformed_df, transformer

In [ ]:
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
columns_to_scale = df.select_dtypes(exclude=['object']).drop(columns=['Exited'], axis=1).columns.tolist()
df_transformed, transformer = apply_transformers(df.drop(columns=['Exited'], axis=1), categorical_columns, columns_to_scale)
df_transformed['Exited'] = df['Exited']

In [ ]:
with open('/transformer.pkl', 'wb') as file:
    pickle.dump(transformer, file)

In [ ]:
X = transformed_df.drop('Exited', axis=1)
y = transformed_df['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
]
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
loss=tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [ ]:
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [ ]:
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8144 - loss: 0.4312 - val_accuracy: 0.8600 - val_loss: 0.3520
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8499 - loss: 0.3573 - val_accuracy: 0.8570 - val_loss: 0.3500
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8583 - loss: 0.3451 - val_accuracy: 0.8655 - val_loss: 0.3364
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8628 - loss: 0.3360 - val_accuracy: 0.8580 - val_loss: 0.3423
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8579 - loss: 0.3444 - val_accuracy: 0.8600 - val_loss: 0.3396
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8564 - loss: 0.3499 - val_accuracy: 0.8580 - val_loss: 0.3415
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8647 - loss: 0.3341 - val_accuracy: 0.8640 - val_loss: 0.3422
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8600 - loss: 0.3443 - val_accu

In [ ]:
model.save('./model.keras')